# LangChain-Expression-Language

- Author: [Suhyun Lee](https://github.com/suhyun0115)
- Design: 
- Peer Review:
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/99-TEMPLATE/00-BASE-TEMPLATE-EXAMPLE.ipynb) [![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/99-TEMPLATE/00-BASE-TEMPLATE-EXAMPLE.ipynb)

## Overview

`RunnablePassthrough` is a tool that **passes data through unchanged** or adds minimal information to it before forwarding. The `invoke()` method of this class **returns the input data without any modifications**.

This enables data to flow to the next stage without being altered.

It is commonly used in conjunction with `RunnableParallel`, which handles multiple tasks simultaneously, and it helps attach new **labels (keys)** to the data.

`RunnablePassthrough` is useful in scenarios such as:

- When there’s no need to transform or modify the data.
- To skip specific stages in a pipeline.
- For debugging or testing, to verify smooth data flow.

In this tutorial, we will implement this using the GPT-4o-mini model and Ollama, based on the LLaMA 3.2 1B model.

### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [Passing Data with RunnablePassthrough and RunnableParallel](#passing-data-with-runnablepassthrough-and-runnableparallel)
  - [Example of Using `RunnableParallel` and `RunnablePassthrough`](#example-of-using-runnableparallel-and-runnablepassthrough)
  - [Summary of Results](#summary-of-results)
- [Search Engine Integration](#search-engine-integration)
  - [Using GPT](#using-gpt)
  - [Using Ollama](#using-ollama)
    - [Ollama Installation Guide on Colab](#ollama-installation-guide-on-colab)

----

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [ ]:
%%capture --no-stderr
!pip install langchain-opentutorial

In [ ]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain_openai",
        "langchain_core",
        "langchain-ollama",
        "langchain_community",
        "faiss-cpu",
    ],
    verbose=False,
    upgrade=False,
)

If you want to get automated tracing of your model calls you can also set your LangSmith API key by uncommenting below code:

In [ ]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "LangChain-Expression-Language",
    }
)

You can alternatively set API keys such as `OPENAI_API_KEY` in a `.env` file and load them.

[Note] This is not necessary if you've already set the required API keys in previous steps.

In [6]:
# Load API keys from .env file
from dotenv import load_dotenv

load_dotenv(override=True)

True

## Passing Data with RunnablePassthrough and RunnableParallel

`RunnablePassthrough` is a tool that **passes data through unchanged** or adds minimal information to it before forwarding.

It is often used with `RunnableParallel` to store data under a new name.

- **Using it alone**
  
  When used on its own, `RunnablePassthrough()` returns the input data as is.

- **Using with `assign`**
  
  When used with `assign` like `RunnablePassthrough.assign(...)`, it adds additional information to the input data before passing it on.

By using `RunnablePassthrough`, you can pass data to the next stage unchanged while adding only the necessary information.

### Example of Using `RunnableParallel` and `RunnablePassthrough`

While `RunnablePassthrough` is useful on its own, it becomes even more powerful when used in combination with `RunnableParallel`.

In this section, we’ll learn how to define and execute **multiple tasks simultaneously** using the `RunnableParallel` class. The step-by-step guide ensures that even beginners can follow along easily.

---

1. **Create a `RunnableParallel` Instance**
   
   First, create an object using the `RunnableParallel` class to execute multiple tasks simultaneously.

2. **Add a `passed` Task**
   
   - Add a task named `passed` that uses `RunnablePassthrough`.
   - This task **returns the input data unchanged**.

3. **Add an `extra` Task**
   
   - Add a task named `extra` that uses `RunnablePassthrough.assign()`.
   - This task multiplies the "num" value in the input data by 3 and stores it under a new key named "mult".

4. **Add a `modified` Task**
   
   - Add a task named `modified` that uses a simple function.
   - This function adds 1 to the "num" value in the input data.

5. **Execute the Tasks**
   
   - After setting up all the tasks, call `runnable.invoke()`.
   - For example, if you input `{"num": 1}`, all the tasks you defined will execute simultaneously.

In [1]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    # Sets up a Runnable that returns the input as-is.
    passed=RunnablePassthrough(),
    # Sets up a Runnable that multiplies the "num" value in the input by 3 and returns the result.
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    # Sets up a Runnable that adds 1 to the "num" value in the input and returns the result.
    modified=lambda x: {"num": x["num"] + 1},
)

# Execute the Runnable with {"num": 1} as input.
result = runnable.invoke({"num": 1})

# Print the result.
print(result)

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

In [2]:
r = RunnablePassthrough.assign(mult=lambda x: x["num"] * 3)
r.invoke({"num": 1})

{'num': 1, 'mult': 3}

### Summary of Results

When the input data is set to `{"num": 1}`, the results of each task are as follows:

1. **`passed`:** Returns the input data unchanged.
   - Result: `{"num": 1}`

2. **`extra`:** Adds a `"mult"` key to the input data, with its value being the `"num"` value multiplied by 3.
   - Result: `{"num": 1, "mult": 3}`

3. **`modified`:** Adds 1 to the `"num"` value.
   - Result: `{"num": 2}`

## Search Engine Integration

The example below demonstrates a use case where `RunnablePassthrough` is utilized.

### Using GPT

In [7]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Create a FAISS vector store from text data.
vectorstore = FAISS.from_texts(
    [
        "Cats are geniuses at claiming boxes as their own.",
        "Dogs have successfully trained humans to take them for walks.",
        "Cats aren't fond of water, but the water in a human's cup is an exception.",
        "Dogs follow cats around, eager to befriend them.",
        "Cats consider laser pointers their arch-nemesis.",
    ],
    embedding=OpenAIEmbeddings(),
)

# Use the vector store as a retriever.
retriever = vectorstore.as_retriever()

# Define a template.
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

# Create a chat prompt from the template.
prompt = ChatPromptTemplate.from_template(template)

In [8]:
# Initialize the ChatOpenAI model.
model = ChatOpenAI(model_name="gpt-4o-mini")


# Function to format retrieved documents.
def format_docs(docs):
    return "\n".join([doc.page_content for doc in docs])


# Construct the retrieval chain.
retrieval_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [9]:
# Execute the retrieval chain to get an answer to a question.
retrieval_chain.invoke("What kind of objects do cats like?")

'Cats like boxes.'

In [10]:
# Execute the retrieval chain to get an answer to a question.
retrieval_chain.invoke("What do dogs like?")

'Dogs like to befriend cats.'

### Using Ollama

- Install the program from the [Ollama official website](https://ollama.com/).
- For detailed information about Ollama, refer to the [GitHub tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/04-Model/10-Ollama.ipynb).
- The `llama3.2` 1b model is used for generating responses, while `mxbai-embed-large` is used for embedding tasks.


### Ollama Installation Guide on Colab

Google Colab does not natively support terminal access, but you can enable it using the `colab-xterm` extension. Below is a step-by-step guide for installing Ollama on Colab.

---

1. **Install and Load `colab-xterm`**
    ```python
   !pip install colab-xterm
   %load_ext colabxterm

2. **Open the Terminal**
    %xterm

3. **Install Ollama**

    In the terminal window that opens, run the following command to install Ollama:
    ```python
    curl -fsSL https://ollama.com/install.sh | sh

4. **Verify Installation** 


   After installation, type ollama in the terminal to check the installation status. If installed correctly, you should see the "Available Commands" list.
    ```python
    ollama

Download and Prepare the Embedding Model for Ollama

In [12]:
!ollama pull mxbai-embed-large

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 819c2adf5ce6...   0% ▕                ▏    0 B/669 MB                  pulling manifest 
pulling 819c2adf5ce6...   0% ▕                ▏    0 B/669 MB                  pulling manifest 
pulling 819c2adf5ce6...   0% ▕                ▏    0 B/669 MB                  pulling manifest 
pulling 819c2adf5ce6...   0% ▕                ▏    0 B/669 MB                  pulling manifest 
pulling 819c2adf5ce6...   0% ▕                ▏    0 B/669 MB                  pulling manifest 
pulling 819c2adf5ce6...   1% ▕                ▏ 4.0 MB/669 MB                  pulling manifest 
pulling 819c2adf5ce6...   1% ▕                ▏ 7.2 MB/669 MB                  pulling manifest 

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import OllamaEmbeddings

# Embedding Configuration
ollama_embeddings = OllamaEmbeddings(model="mxbai-embed-large")

# Create a FAISS vector store from text data.
vectorstore = FAISS.from_texts(
    [
        "Cats are geniuses at claiming boxes as their own.",
        "Dogs have successfully trained humans to take them for walks.",
        "Cats aren't fond of water, but the water in a human's cup is an exception.",
        "Dogs follow cats around, eager to befriend them.",
        "Cats consider laser pointers their arch-nemesis.",
    ],
    embedding=ollama_embeddings(),
)
# Use the vector store as a retriever.
retriever = vectorstore.as_retriever()

# Define a template.
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

# Create a chat prompt from the template.
prompt = ChatPromptTemplate.from_template(template)

Download and Prepare the Model for Answer Generation

In [11]:
!ollama pull llama3.2:1b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 74701a8c35f6...   0% ▕                ▏    0 B/1.3 GB                  pulling manifest 
pulling 74701a8c35f6...   0% ▕                ▏    0 B/1.3 GB                  pulling manifest 
pulling 74701a8c35f6...   0% ▕                ▏    0 B/1.3 GB                  pulling manifest 
pulling 74701a8c35f6...   0% ▕                ▏    0 B/1.3 GB                  pulling manifest 
pulling 74701a8c35f6...   0% ▕                ▏    0 B/1.3 GB                  pulling manifest 
pulling 74701a8c35f6...   0% ▕                ▏    0 B/1.3 GB                  pulling manifest 
pulling 74701a8c35f6...   0% ▕                ▏    0 B/1.3 GB                  pulling manifest 

In [15]:
from langchain_ollama import ChatOllama

ollama_model = ChatOllama(model="llama3.2:1b")


# Function to format retrieved documents.
def format_docs(docs):
    return "\n".join([doc.page_content for doc in docs])


# Construct the retrieval chain.
retrieval_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | ollama_model  # Switch to the Ollama model
    | StrOutputParser()
)

In [16]:
# Execute the retrieval chain to get an answer to a question.
retrieval_chain.invoke("What kind of objects do cats like?")

'Based on this context, it seems that cats tend to enjoy and claim boxes as their own.'

In [ ]:
# Execute the retrieval chain to get an answer to a question.
retrieval_chain.invoke("What do dogs like?")

'Based on the context, it seems that dogs enjoy being around cats and having them follow them. Additionally, dogs have successfully trained humans to take them for walks.'